# Scrapping LinkedIn profiles

In [430]:
import os
import requests
import re
import pandas as pd
from requests.exceptions import RequestException
import json
from pandas import json_normalize
import datetime

# Function that scrapes LinkedIn profiles

In [686]:
def scrape_linkedin_profile2(linkedin_profile_url: str):
    api_key = "2ZjHkun0Sq6KGa_o-4p7jg"
    header_dic = {"Authorization": f"Bearer {api_key}"}
    api_endpoint = "https://nubela.co/proxycurl/api/v2/linkedin"

    response = requests.get(
        api_endpoint, params={"url": linkedin_profile_url}, headers=header_dic
    )

    try:
        data = response.json()
        data = {
            k: v
            for k, v in data.items()
            if v not in ([], "", "", None)
            and k
            not in [
                "people_also_viewed",
                "recommendations",
                "similarly_named_profiles",
                "articles",
                "background_cover_image_url",
                "activities",
                "volunteer_work",
            ]
        }
        if data.get("groups"):
            for group_dict in data.get("groups"):
                group_dict.pop("profile_pic_url")

        return data

    except json.JSONDecodeError:

        return None



# Function that scrapes company's profiles

In [752]:
def scrape_linkedin_company_data(linkedin_company_url):
    api_key = "2ZjHkun0Sq6KGa_o-4p7jg"
    headers = {'Authorization': f'Bearer {api_key}'}

    api_endpoint_1 = 'https://nubela.co/proxycurl/api/linkedin/company'
    params_1 = {
        'url': linkedin_company_url,
        'use_cache': 'if-present',
    }
    response_1 = requests.get(api_endpoint_1, params=params_1, headers=headers)

    if response_1.status_code == 200:
        data = response_1.json()
        return data
    
    else:
        
        api_endpoint_2 = 'https://nubela.co/proxycurl/api/linkedin/school'
        params_2 = {
            'url': linkedin_company_url,  # Utiliza la misma URL
            'use_cache': 'if-present',
        }
        response_2 = requests.get(api_endpoint_2, params=params_2, headers=headers)

        if response_2.status_code == 200:
            data = response_2.json()
            return data
        else:
            print(f"Error al obtener datos de {linkedin_company_url} con la segunda opción.")
            return None

# Scraping and cleaning our dataframes

## First dataframe: 2015-2016 class

In [690]:
df1 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2015-2016.csv') 

df1 = df1[df1['Link perfil'] != 'Link no existente']

df1 = df1.reset_index()

df1 = df1.drop('index', axis=1)

'''We open our CSV, containing the full name and URL of the profile of our students, and we remove all those
rows that don't contain the URL of the student, because our Selenium code couldn't scrap it'''

"We open our CSV, containing the full name and URL of the profile of our students, and we remove all those\nrows that don't contain the URL of the student, because our Selenium code couldn't scrap it"

In [691]:
df1["Datos escrapeados"] = df1["Link perfil"].apply(scrape_linkedin_profile2)

# We apply our function to the row of our dataframe that contains all the URLs of our students


In [693]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df1.loc[df1['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df1 = df1.drop(indices_a_eliminar)

# And we take them out 


In [695]:
df1 = df1.reset_index()

df1 = df1.drop('index', axis=1)

# We reset indexes and delete the column 'index'

In [696]:
normalized_data = pd.json_normalize(df1['Datos escrapeados']) 

# We create one column per key in our 'Datos escrapeados' column


In [697]:
df1 = pd.concat([df1, normalized_data], axis=1) # We apply those columns to our df1 dataframe


In [699]:
columnas_a_eliminar = ['Nombre', 'Primer apellido', 'Segundo apellido', 'Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']


df1 = df1.drop(columns=columnas_a_eliminar)


In [700]:
df1 = df1.explode('experiences')

# We explode our lists in 'experiences' column, to have as many rows per student as experiences she/he has

In [701]:
df1 = df1.reset_index()

df1 = df1.drop('index', axis=1)

# We reset our indexes, to make it more clear

In [702]:
normalized_data1 = pd.json_normalize(df1['experiences']) 


df1 = pd.concat([df1, normalized_data1], axis=1) # We apply those columns to our df1 dataframe


In [704]:
cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'experiences']                  
                   
df1 = df1.drop(cols_a_eliminar, axis=1)

# We get rid of unwanted columns

In [759]:
copi1 = df1.copy() # We create a copy of our dataframe just in case


In [760]:
copi1 = copi1[copi1["company_linkedin_profile_url"].notna()]

In [761]:
copi1 = copi1.reset_index()
copi1 = copi1.drop('index', axis=1)

copi1 

# We reset indexes

,Grado,Nombre completo,Link perfil,city,company,company_linkedin_profile_url,title,location,starts_at.day,starts_at.month,starts_at.year,ends_at.day,ends_at.month,ends_at.year
0,Comunicación audiovisual y Publicidad,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Madrid,TBWA\ España,https://es.linkedin.com/company/tbwa-spain,Junior Art Director/ Crearive,None,1.0,10.0,2018.0,31.0,7.0,2019.0
1,Comunicación audiovisual y Publicidad,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Madrid,ContrapuntoBBDO,https://es.linkedin.com/company/contra-punto-bbdo,Junior Art Director / Creative,None,1.0,2.0,2018.0,30.0,9.0,2018.0
2,Comunicación audiovisual y Publicidad,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Madrid,Publicis Groupe,https://es.linkedin.com/company/publicis-groupe,Junior Art Director / Creative,Madrid,1.0,6.0,2016.0,31.0,12.0,2016.0
3,Comunicación audiovisual y Publicidad,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Madrid,Längstrump,https://es.linkedin.com/company/langstrump,Junior Art Director / Creative,Madrid,1.0,9.0,2015.0,29.0,2.0,2016.0
4,Comunicación audiovisual y Publicidad,Antonio Cortés Ruiz,https://www.linkedin.com/in/antonio-cort%C3%A9...,Madrid,Grupo Boomerang TV,https://es.linkedin.com/company/boomerang-tv,Ayudante de Producción,Madrid,1.0,4.0,2015.0,31.0,7.0,2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,Comunicación digital,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,Madrid,Santander Global Operations,https://es.linkedin.com/company/santander-glob...,Digital Content Manager para Banco Santander (...,"Madrid, Comunidad de Madrid, España",1.0,10.0,2021.0,NaN,NaN,NaN
1010,Comunicación digital,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,Madrid,easylife conciliación,https://es.linkedin.com/company/easylife-conci...,Gestor de proyectos - Carta de Beneficios,"Madrid, Comunidad de Madrid, España",1.0,12.0,2020.0,30.0,9.0,2021.0
1011,Comunicación digital,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,Madrid,easylife conciliación,https://es.linkedin.com/company/easylife-conci...,Especialista en comunicación corporativa y org...,Madrid y alrededores,1.0,12.0,2016.0,30.0,6.0,2021.0
1012,Comunicación digital,Alvaro Diaz Fernandez,https://www.linkedin.com/in/alvaro-diaz-fdez?m...,Madrid,Grupo Trivium IC,https://es.linkedin.com/company/grupo-trivium-ic,Redactor de contenidos web,"Gran Vía 33, Madrid",1.0,9.0,2015.0,31.0,1.0,2016.0


In [762]:
copi1["Datos empresa"] = copi1["company_linkedin_profile_url"].apply(scrape_linkedin_company_data)


Error al obtener datos de https://www.linkedin.com/company/18505126/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/40655773/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/1675/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/1675/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/163929/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/2558499/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/2558499/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/2558499/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/15092653/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/9472/ con la segunda opción.
Error al obtener datos de https://es.linkedin.com/company/private-tutor con la segunda opción.
Er

KeyboardInterrupt: 

In [721]:
df2['ends_at.day'].fillna('actualidad', inplace=True)
df2['ends_at.month'].fillna('actualidad', inplace=True)
df2['ends_at.year'].fillna('actualidad', inplace=True)

# If any of our ending columns have nulls, it means that that person is still working there

In [722]:
df2.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

# We get rid of all our experiences whose start date is null

In [723]:

# Combines the columns of start date and end date into 'end date'
df2['Fecha inicio'] = df2.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
df2['Fecha fin'] = df2.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

# Calculates the duration of each experience in months
df2['Duración (meses)'] = df2.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

# Replaces values in end date when its pertinent
df2['Fecha fin'] = df2.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

# Gets rid of unnecesary columns
df2.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)


In [716]:
df2.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

# We rename some columns in spanish

In [717]:
df2 = df2.reset_index()

df2 = df2.drop('index', axis=1)

In [718]:
new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

df2 = df2[new_order]

# We finally change the orders of some columns to make them more comprehensible


In [568]:
df2.to_csv('/Users/davidfernandez/Desktop/clean/experience/2015-2016.csv', index=False) 


## Second dataframe: 2016-2017 class

In [592]:
df2 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2016-2017.csv') 

df2 = df2[df2['Link perfil'] != 'Link no existente']

df2 = df2.reset_index()

df2 = df2.drop('index', axis=1)

In [593]:
df2["Datos escrapeados"] = df2["Link perfil"].apply(scrape_linkedin_profile2)


In [595]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df2.loc[df2['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df2 = df2.drop(indices_a_eliminar)

# And we take them out 

In [597]:
copi = df2.copy()

In [598]:
copi = copi.reset_index()
copi = copi.drop('index', axis=1)

normalized_data = pd.json_normalize(copi['Datos escrapeados'])

copi = pd.concat([copi, normalized_data], axis=1)

columnas_a_eliminar = ['Nombre', 'Primer apellido', 'Segundo apellido', 'Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi = copi.drop(columns=columnas_a_eliminar)

copi = copi.explode('experiences')

copi = copi.reset_index()
copi = copi.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi['experiences'])

copi = pd.concat([copi, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'company_linkedin_profile_url', 'experiences']

copi = copi.drop(cols_a_eliminar, axis=1)


In [599]:
copi['ends_at.day'].fillna('actualidad', inplace=True)
copi['ends_at.month'].fillna('actualidad', inplace=True)
copi['ends_at.year'].fillna('actualidad', inplace=True)

copi.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi['Fecha inicio'] = copi.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi['Fecha fin'] = copi.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi['Duración (meses)'] = copi.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi['Fecha fin'] = copi.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi = copi.reset_index()
copi = copi.drop('index', axis=1)

new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copi = copi[new_order]


In [601]:
copi.to_csv('/Users/davidfernandez/Desktop/clean/experience/2016-2017.csv', index=False) 


## Third dataframe: 2017-2018 class

In [602]:
df3 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2017-2018.csv') 

df3 = df3[df3['Link perfil'] != 'Link no existente']

df3 = df3.reset_index()

df3 = df3.drop('index', axis=1)

In [604]:
df3["Datos escrapeados"] = df3["Link perfil"].apply(scrape_linkedin_profile2)


In [606]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df3.loc[df3['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df3 = df3.drop(indices_a_eliminar)

# And we take them out 

In [616]:
copi2 = df3.copy()

In [617]:
copi2 = copi2.reset_index()
copi2 = copi2.drop('index', axis=1)

normalized_data = pd.json_normalize(copi2['Datos escrapeados'])

copi2 = pd.concat([copi2, normalized_data], axis=1)

columnas_a_eliminar = ['Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi2 = copi2.drop(columns=columnas_a_eliminar)

copi2 = copi2.explode('experiences')

copi2 = copi2.reset_index()
copi2 = copi2.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi2['experiences'])

copi2 = pd.concat([copi2, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'company_linkedin_profile_url', 'experiences']

copi2 = copi2.drop(cols_a_eliminar, axis=1)


In [618]:
copi2['ends_at.day'].fillna('actualidad', inplace=True)
copi2['ends_at.month'].fillna('actualidad', inplace=True)
copi2['ends_at.year'].fillna('actualidad', inplace=True)

copi2.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi2['Fecha inicio'] = copi2.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi2['Fecha fin'] = copi2.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi2['Duración (meses)'] = copi2.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi2['Fecha fin'] = copi2.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi2.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi2.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi2 = copi2.reset_index()
copi2 = copi2.drop('index', axis=1)

new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copi2 = copi2[new_order]


In [620]:
copi2.to_csv('/Users/davidfernandez/Desktop/clean/experience/2017-2018.csv', index=False)

## Fifth dataframe: 2019-2020 class

In [626]:
df5 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2019-2020.csv')

df5 = df5[df5['Link perfil'] != 'Link no existente']

df5 = df5.reset_index()

df5 = df5.drop('index', axis=1)

In [627]:
df5["Datos escrapeados"] = df5["Link perfil"].apply(scrape_linkedin_profile2)

In [629]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df5.loc[df5['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df5 = df5.drop(indices_a_eliminar)

# And we take them out 


In [630]:
copi3 = df5.copy()

In [631]:
copi3 = copi3.reset_index()
copi3 = copi3.drop('index', axis=1)

normalized_data = pd.json_normalize(copi3['Datos escrapeados'])

copi3 = pd.concat([copi3, normalized_data], axis=1)

columnas_a_eliminar = ['Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi3 = copi3.drop(columns=columnas_a_eliminar)

copi3 = copi3.explode('experiences')

copi3 = copi3.reset_index()
copi3 = copi3.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi3['experiences'])

copi3 = pd.concat([copi3, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'company_linkedin_profile_url', 'experiences']

copi3 = copi3.drop(cols_a_eliminar, axis=1)


In [632]:
copi3['ends_at.day'].fillna('actualidad', inplace=True)
copi3['ends_at.month'].fillna('actualidad', inplace=True)
copi3['ends_at.year'].fillna('actualidad', inplace=True)

copi3.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi3['Fecha inicio'] = copi3.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi3['Fecha fin'] = copi3.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi3['Duración (meses)'] = copi3.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi3['Fecha fin'] = copi3.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi3.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi3.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi3 = copi3.reset_index()
copi3 = copi3.drop('index', axis=1)

new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copi3 = copi3[new_order]



In [634]:
copi3.to_csv('/Users/davidfernandez/Desktop/clean/experience/2019-2020.csv', index=False)

## Sixth dataframe: 2020-2021 class

In [635]:
df6 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2020-2021.csv')

df6 = df6[df6['Link perfil'] != 'Link no existente']

df6 = df6.reset_index()

df6 = df6.drop('index', axis=1)

In [637]:
df6["Datos escrapeados"] = df6["Link perfil"].apply(scrape_linkedin_profile2)

In [639]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df6.loc[df6['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df6 = df6.drop(indices_a_eliminar)

# And we take them out 


In [640]:
copi4 = df6.copy()

In [641]:
copi4 = copi4.reset_index()
copi4 = copi4.drop('index', axis=1)

normalized_data = pd.json_normalize(copi4['Datos escrapeados'])

copi4 = pd.concat([copi4, normalized_data], axis=1)

columnas_a_eliminar = ['Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi4 = copi4.drop(columns=columnas_a_eliminar)

copi4 = copi4.explode('experiences')

copi4 = copi4.reset_index()
copi4 = copi4.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi4['experiences'])

copi4 = pd.concat([copi4, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'company_linkedin_profile_url', 'experiences']

copi4 = copi4.drop(cols_a_eliminar, axis=1)


In [642]:
copi4['ends_at.day'].fillna('actualidad', inplace=True)
copi4['ends_at.month'].fillna('actualidad', inplace=True)
copi4['ends_at.year'].fillna('actualidad', inplace=True)

copi4.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi4['Fecha inicio'] = copi4.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi4['Fecha fin'] = copi4.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi4['Duración (meses)'] = copi4.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi4['Fecha fin'] = copi4.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi4.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi4.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi4 = copi4.reset_index()
copi4 = copi4.drop('index', axis=1)

new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copi4 = copi4[new_order]


In [644]:
copi4.to_csv('/Users/davidfernandez/Desktop/clean/experience/2020-2021.csv', index=False)

## Seventh dataframe: 2021-2022 class

In [645]:
df7 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2021-2022.csv')

df7 = df7[df7['Link perfil'] != 'Link no existente']

df7 = df7.reset_index()

df7 = df7.drop('index', axis=1)

In [647]:
df7["Datos escrapeados"] = df7["Link perfil"].apply(scrape_linkedin_profile2)

In [649]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df7.loc[df7['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df7 = df7.drop(indices_a_eliminar)

# And we take them out 

In [654]:
copi5 = df7.copy()

In [656]:
copi5 = copi5.reset_index()
copi5 = copi5.drop('index', axis=1)

normalized_data = pd.json_normalize(copi5['Datos escrapeados'])

copi5 = pd.concat([copi5, normalized_data], axis=1)

columnas_a_eliminar = ['Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi5 = copi5.drop(columns=columnas_a_eliminar)

copi5 = copi5.explode('experiences')

copi5 = copi5.reset_index()
copi5 = copi5.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi5['experiences'])

copi5 = pd.concat([copi5, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'company_linkedin_profile_url', 'experiences']

copi5 = copi5.drop(cols_a_eliminar, axis=1)


In [658]:
copi5['ends_at.day'].fillna('actualidad', inplace=True)
copi5['ends_at.month'].fillna('actualidad', inplace=True)
copi5['ends_at.year'].fillna('actualidad', inplace=True)

copi5.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi5['Fecha inicio'] = copi5.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi5['Fecha fin'] = copi5.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi5['Duración (meses)'] = copi5.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi5['Fecha fin'] = copi5.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi5.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi5.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi5 = copi5.reset_index()
copi5 = copi5.drop('index', axis=1)

new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copi5 = copi5[new_order]


In [660]:
copi5.to_csv('/Users/davidfernandez/Desktop/clean/experience/2021-2022.csv', index=False)

## Eight dataframe: 2022-2023 class

In [661]:
df8 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2022-2023.csv')

df8 = df8[df8['Link perfil'] != 'Link no existente']

df8 = df8.reset_index()

df8 = df8.drop('index', axis=1)

In [664]:
df8["Datos escrapeados"] = df8["Link perfil"].apply(scrape_linkedin_profile2)

In [666]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df8.loc[df8['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df8 = df8.drop(indices_a_eliminar)

# And we take them out 

In [679]:
copi6 = df8.copy()

In [680]:
copi6 = copi6.reset_index()
copi6 = copi6.drop('index', axis=1)

normalized_data = pd.json_normalize(copi6['Datos escrapeados'])

copi6 = pd.concat([copi6, normalized_data], axis=1)

columnas_a_eliminar = ['Email universitario', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi6 = copi6.drop(columns=columnas_a_eliminar)

copi6 = copi6.explode('experiences')

copi6 = copi6.reset_index()
copi6 = copi6.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi6['experiences'])

copi6 = pd.concat([copi6, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'company_linkedin_profile_url', 'experiences']

copi6 = copi6.drop(cols_a_eliminar, axis=1)


In [682]:
copi6['ends_at.day'].fillna('actualidad', inplace=True)
copi6['ends_at.month'].fillna('actualidad', inplace=True)
copi6['ends_at.year'].fillna('actualidad', inplace=True)

copi6.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi6['Fecha inicio'] = copi6.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi6['Fecha fin'] = copi6.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi6['Duración (meses)'] = copi6.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi6['Fecha fin'] = copi6.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi6.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi6.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi6 = copi6.reset_index()
copi6 = copi6.drop('index', axis=1)

new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copi6 = copi6[new_order]


In [684]:
copi6.to_csv('/Users/davidfernandez/Desktop/clean/experience/2022-2023.csv', index=False)